In [47]:
import pandas as pd

df = pd.read_csv('famil_family.csv')

df.fillna(' ')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4119 entries, 0 to 4118
Data columns (total 10 columns):
Unnamed: 0           4119 non-null int64
Department:          4113 non-null object
Position Title:      4112 non-null object
POSITION SUMMARY:    3458 non-null object
responsabilities     3815 non-null object
EI                   2763 non-null object
CA                   3142 non-null object
CP                   2436 non-null object
TS                   3385 non-null object
family               4113 non-null object
dtypes: int64(1), object(9)
memory usage: 321.9+ KB


In [48]:
family=df[df['family'] != 'project plannersschedulers' ]

In [49]:
family_list= df['family'].unique().tolist()
family_dict={}

for i in range(len(family_list)): family_dict[family_list[i]] = i
    
family['label'] = df['family'].map(family_dict)





c:\users\rushi\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [50]:
print(family.info())
family['train'] = family['Department: '] + family['Position Title:'] + family['POSITION SUMMARY:'] + family['responsabilities'] + family['EI'] + family['CA'] + family['CP'] + family['TS'] 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2053 entries, 0 to 4116
Data columns (total 11 columns):
Unnamed: 0           2053 non-null int64
Department:          2047 non-null object
Position Title:      2049 non-null object
POSITION SUMMARY:    1693 non-null object
responsabilities     1975 non-null object
EI                   1467 non-null object
CA                   1703 non-null object
CP                   1230 non-null object
TS                   1726 non-null object
family               2047 non-null object
label                2053 non-null int64
dtypes: int64(2), object(9)
memory usage: 192.5+ KB
None


c:\users\rushi\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [51]:
family=family.dropna()

In [55]:
family['train']

family['train'] = family['train'].apply(lambda x: " ".join(x.lower() for x in x.split()))
family['train'] = family['train'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
stop = stopwords.words('english')

family['train'] = family['train'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

rfreq = pd.Series(' '.join(family['train']).split()).value_counts()[-10:]
print("\n\nrare words and count:\n",rfreq,'\n\n')

####
family['train']=family['train'].apply(lambda x: " ".join(x for x in x.split() if x not in rfreq))






rare words and count:
 operationspossess       1
heterogeneity           1
photos                  1
excelcreative           1
changesproven           1
engineeringcorrosion    1
yrsworking              1
deadlinesknowledge      1
productionhusky         1
campworkforce           1
dtype: int64 




In [57]:
from sklearn.externals import joblib
train=family.sample(frac=0.8,random_state=200)
test=family.drop(train.index)

train_X = train['train']
train_Y = train['label']
test_X =  test ['train']
test_Y =  test['label']

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(train_X)
print('cool')
joblib.dump(vectorizer,'family_clf_tfidfmodel.mod')
test_vectors = vectorizer.transform(test_X)

print(train_vectors.shape, test_vectors.shape)




cool
(758, 15107) (189, 15107)


In [ ]:
import tensorflow as tf


tf.app.flags.DEFINE_integer('training_iteration', 1000,
                            'number of training iterations.')
tf.app.flags.DEFINE_integer('model_version', 1, 'version number of the model.')
tf.app.flags.DEFINE_string('work_dir', '/tmp', 'Working directory.')
FLAGS = tf.app.flags.FLAGS


def main(_):
  if len(sys.argv) < 2 or sys.argv[-1].startswith('-'):
    print('Usage: mnist_saved_model.py [--training_iteration=x] '
          '[--model_version=y] export_dir')
    sys.exit(-1)
  if FLAGS.training_iteration <= 0:
    print('Please specify a positive value for training iteration.')
    sys.exit(-1)
  if FLAGS.model_version <= 0:
    print('Please specify a positive value for version number.')
    sys.exit(-1)

  # Train model
  print('Training model...')
  mnist = mnist_input_data.read_data_sets(FLAGS.work_dir, one_hot=True)
  sess = tf.InteractiveSession()
  serialized_tf_example = tf.placeholder(tf.string, name='tf_example')
  feature_configs = {'x': tf.FixedLenFeature(shape=[784], dtype=tf.float32),}
  tf_example = tf.parse_example(serialized_tf_example, feature_configs)
  x = tf.identity(tf_example['x'], name='x')  # use tf.identity() to assign name
  y_ = tf.placeholder('float', shape=[None, 10])
  w = tf.Variable(tf.zeros([784, 10]))
  b = tf.Variable(tf.zeros([10]))
  sess.run(tf.global_variables_initializer())
  y = tf.nn.softmax(tf.matmul(x, w) + b, name='y')
  cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
  train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
  values, indices = tf.nn.top_k(y, 10)
  table = tf.contrib.lookup.index_to_string_table_from_tensor(
      tf.constant([str(i) for i in range(10)]))
  prediction_classes = table.lookup(tf.to_int64(indices))
  for _ in range(FLAGS.training_iteration):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})
  correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
  print('training accuracy %g' % sess.run(
      accuracy, feed_dict={
          x: mnist.test.images,
          y_: mnist.test.labels
      }))
  print('Done training!')

  # Export model
  # WARNING(break-tutorial-inline-code): The following code snippet is
  # in-lined in tutorials, please update tutorial documents accordingly
  # whenever code changes.
  export_path_base = sys.argv[-1]
  export_path = os.path.join(
      tf.compat.as_bytes(export_path_base),
      tf.compat.as_bytes(str(FLAGS.model_version)))
  print('Exporting trained model to', export_path)
  builder = tf.saved_model.builder.SavedModelBuilder(export_path)

  # Build the signature_def_map.
  classification_inputs = tf.saved_model.utils.build_tensor_info(
      serialized_tf_example)
  classification_outputs_classes = tf.saved_model.utils.build_tensor_info(
      prediction_classes)
  classification_outputs_scores = tf.saved_model.utils.build_tensor_info(values)

  classification_signature = (
      tf.saved_model.signature_def_utils.build_signature_def(
          inputs={
              tf.saved_model.signature_constants.CLASSIFY_INPUTS:
                  classification_inputs
          },
          outputs={
              tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES:
                  classification_outputs_classes,
              tf.saved_model.signature_constants.CLASSIFY_OUTPUT_SCORES:
                  classification_outputs_scores
          },
          method_name=tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME))

  tensor_info_x = tf.saved_model.utils.build_tensor_info(x)
  tensor_info_y = tf.saved_model.utils.build_tensor_info(y)

  prediction_signature = (
      tf.saved_model.signature_def_utils.build_signature_def(
          inputs={'images': tensor_info_x},
          outputs={'scores': tensor_info_y},
          method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

  builder.add_meta_graph_and_variables(
      sess, [tf.saved_model.tag_constants.SERVING],
      signature_def_map={
          'predict_images':
              prediction_signature,
          tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
              classification_signature,
      },
      main_op=tf.tables_initializer(),
      strip_default_attrs=True)

  builder.save()

  print('Done exporting!')


if __name__ == '__main__':
  tf.app.run()

In [70]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(train_X, train_Y)

joblib.dump(nb,'nb_family_classifier.model')

from sklearn.metrics import classification_report
y_pred = nb.predict(test_X)

print('accuracy %s' % accuracy_score(y_pred, test_Y))
print(classification_report(test_Y, y_pred))

accuracy 0.3915343915343915
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       1.00      0.04      0.08        24
           3       0.94      0.48      0.64        33
           4       0.00      0.00      0.00        17
           5       0.00      0.00      0.00        10
           6       0.00      0.00      0.00        10
           8       0.28      1.00      0.44        45
           9       0.00      0.00      0.00         8
          10       0.00      0.00      0.00         3
          18       1.00      0.55      0.71        22
          20       0.00      0.00      0.00         1
          23       0.00      0.00      0.00         4
          35       0.00      0.00      0.00         1
          36       0.00      0.00      0.00         1
          37       0.00      0.00      0.00         5

   micro avg       0.39      0.39      0.39       189
   macro avg       0.21      0.14      0.12       18

c:\users\rushi\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [71]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(train_X, train_Y)

joblib.dump(sgd,'sgd_family_classifier.model')

y_pred = sgd.predict(test_X)


print('accuracy %s' %accuracy_score(y_pred, test_Y))
print(classification_report(test_Y, y_pred))

accuracy 0.8518518518518519
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       0.96      0.92      0.94        24
           3       0.78      0.94      0.85        33
           4       0.87      0.76      0.81        17
           5       0.83      1.00      0.91        10
           6       0.67      0.80      0.73        10
           8       0.84      0.84      0.84        45
           9       0.67      0.50      0.57         8
          10       1.00      0.67      0.80         3
          18       1.00      1.00      1.00        22
          20       1.00      1.00      1.00         1
          23       1.00      0.50      0.67         4
          35       0.00      0.00      0.00         1
          36       0.00      0.00      0.00         1
          37       0.75      0.60      0.67         5

   micro avg       0.85      0.85      0.85       189
   macro avg       0.76      0.70      0.72       18

In [72]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(train_X, train_Y)

joblib.dump(logreg,'logreg_family_classifier.model')

y_pred = logreg.predict(test_X)


print('accuracy %s' % accuracy_score(y_pred, test_Y))
print(classification_report(test_Y, y_pred))

c:\users\rushi\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\rushi\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy 0.8465608465608465
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      0.92      0.96        24
           3       0.78      0.88      0.83        33
           4       0.87      0.76      0.81        17
           5       0.83      1.00      0.91        10
           6       0.62      0.80      0.70        10
           8       0.81      0.87      0.84        45
           9       0.67      0.50      0.57         8
          10       1.00      0.67      0.80         3
          18       1.00      1.00      1.00        22
          20       1.00      1.00      1.00         1
          23       1.00      0.50      0.67         4
          35       0.00      0.00      0.00         1
          36       0.00      0.00      0.00         1
          37       0.75      0.60      0.67         5

   micro avg       0.85      0.85      0.85       189
   macro avg       0.76      0.70      0.72       18